In [1]:
import re

import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score

import math

import nltk

from sklearn.feature_extraction.text import CountVectorizer

from collections import defaultdict

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv
/kaggle/input/d/arkhoshghalb/twitter-sentiment-analysis-hatred-speech/train.csv
/kaggle/input/d/arkhoshghalb/twitter-sentiment-analysis-hatred-speech/test.csv


In [3]:
df=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df['review'][3].lower()

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.<br /><br />ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

In [6]:
df['review']=df['review'].str.lower()

In [7]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


# Remove HTML Tags


In [8]:
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)
    

In [9]:
text = "basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.<br /><br />ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

In [10]:
remove_html_tags(text)

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

In [11]:
df['review'].apply(remove_html_tags)

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. the filming tec...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

# Remove URLs


In [12]:
def remove_url(text):
    pattern = re.compile(r'https?://\$+|www\.\$+')
    return pattern.sub(r'',text)

In [13]:
text1 = 'For NYKA fashion go through https://www.nykaa.com/?utm_content=ads&utm_source=GooglePaid&utm_medium=search&utm_campaign=Search_Nykaa_NCA&hlp=hlpa&gad_source=1&gclid=CjwKCAjwzIK1BhAuEiwAHQmU3lTPusFUDBNEROccLZ3ri2i73G8anmQrTtrhk3kRerJCx2xPkF0mnRoCKMUQAvD_BwE '

text2 = 'Google search here www.google.com'

text3 = """The link of this question: https://stackoverflow.com/questions/6038061/regular-expression-to-find-urls-within-a-string
Also there are some urls: www.google.com, facebook.com, http://test.com/method?param=wasd, http://test.com/method?param=wasd&params2=kjhdkjshd
The code below catches all urls in text and returns urls in list."""

In [14]:
remove_url(text2)

'Google search here www.google.com'

# Remove punctuations - Tokenization


In [15]:
import string,time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [16]:
exclude = string.punctuation

In [17]:
def remove_punctuation(text):
    for char in exclude:
        text = text.replace(char,'')
    return text

In [18]:
text = 'String.With.Punctuation,myfamily) to be * home work// she go through#@and work hard.'

In [19]:
remove_punctuation(text)

'StringWithPunctuationmyfamily to be  home work she go throughand work hard'

In [20]:
import time
start = time.time()
print(start)
print(remove_punctuation(text))
time1 = time.time()-start
print(time1)
print(time1*50000)

1722056363.1053228
StringWithPunctuationmyfamily to be  home work she go throughand work hard
0.00019407272338867188
9.703636169433594


In [21]:
def remove_punct(text):
    return text.translate(str.maketrans('','',exclude))

# Sentiment Analysis


In [22]:
df = pd.read_csv('/kaggle/input/d/arkhoshghalb/twitter-sentiment-analysis-hatred-speech/train.csv')

In [23]:
df.sample(5)

,id,label,tweet
29524,29525,0,@user lots of #sponsors on the table @user @u...
8286,8287,0,nightcap #sandiego #sandiegoliving #darkandsto...
8357,8358,0,sunday everyone - have a good day ðð #...
6316,6317,0,its raining offer at #pantaloons #shop and be
7335,7336,0,here you have it everyone. science :) make mor...


In [24]:
df['tweet']=df['tweet'].apply(remove_punct)

# Chat word treatment


In [25]:
chat_word = {"AFAIK":"As Far As I Know",
"AFK":"Away From Keyboard",
"ASAP":"As Soon As Possible",
"ATK":"At The Keyboard",
"ATM":"At The Moment",
"A3":"Anytime, Anywhere, Anyplace",
"BAK":"Back At Keyboard",
"BBL":"Be Back Later",
"BBS":"Be Back Soon",
"FYI" : "For Your Information",
            "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "B4N": "Bye For Now",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when. TFW internet slang often goes in a caption to an image.",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "LOL": "Laughing out loud",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don’t care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "IDC": "I don’t care",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "LMAO": "Laughing my a** off",
    "BFF": "Best friends forever",
    "CSL": "Can’t stop laughing"}

In [26]:
print(chat_word)

{'AFAIK': 'As Far As I Know', 'AFK': 'Away From Keyboard', 'ASAP': 'As Soon As Possible', 'ATK': 'At The Keyboard', 'ATM': 'At The Moment', 'A3': 'Anytime, Anywhere, Anyplace', 'BAK': 'Back At Keyboard', 'BBL': 'Be Back Later', 'BBS': 'Be Back Soon', 'FYI': 'For Your Information', 'BFN': 'Bye For Now', 'B4N': 'Bye For Now', 'BRB': 'Be Right Back', 'BRT': 'Be Right There', 'BTW': 'By The Way', 'B4': 'Before', 'CU': 'See You', 'CUL8R': 'See You Later', 'CYA': 'See You', 'FAQ': 'Frequently Asked Questions', 'FC': 'Fingers Crossed', 'FWIW': "For What It's Worth", 'GAL': 'Get A Life', 'GG': 'Good Game', 'GN': 'Good Night', 'GMTA': 'Great Minds Think Alike', 'GR8': 'Great!', 'G9': 'Genius', 'IC': 'I See', 'ICQ': 'I Seek you (also a chat program)', 'ILU': 'I Love You', 'IMHO': 'In My Honest/Humble Opinion', 'IMO': 'In My Opinion', 'IOW': 'In Other Words', 'IRL': 'In Real Life', 'KISS': 'Keep It Simple, Stupid', 'LDR': 'Long Distance Relationship', 'LMAO': 'Laughing my a** off', 'LOL': 'Laughi

In [27]:
def chat_conversion(text):
    new_text=[]
    for w in text.split():
        if w.upper() in chat_word:
            new_text.append(chat_word[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [28]:
chat_conversion(" I will complete my work ASAP .")

'I will complete my work As Soon As Possible .'

In [29]:
chat_conversion(" FYI New Delhi is the captial of India")

'For Your Information New Delhi is the captial of India'

# Spelling Correction

In [30]:
from textblob import TextBlob

In [31]:
incorrect_text = 'Ceertain conditon dring severel genrasion are modifyied in thee same mannerr.'
textBlb = TextBlob(incorrect_text)
textBlb.correct().string

'Certain condition during several generation are modified in thee same manner.'

# Remove Stop Words

In [32]:
from nltk.corpus import stopwords

In [33]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [34]:
def remove_stop_word(text):
    new_text = []
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [35]:
df.head()

,id,label,tweet
0,1,0,user when a father is dysfunctional and is so...
1,2,0,user user thanks for lyft credit i cant use ca...
2,3,0,bihday your majesty
3,4,0,model i love u take with u all the time in u...
4,5,0,factsguide society now motivation


In [36]:
df['tweet'].apply(remove_stop_word)

0        user   father  dysfunctional    selfish  drags...
1        user user thanks  lyft credit  cant use cause ...
2                                          bihday  majesty
3        model  love u take  u   time  urð± ððð...
4                           factsguide society  motivation
                               ...                        
31957    ate user isz  youuuððððððð...
31958     see nina turner   airwaves trying  wrap    ma...
31959    listening  sad songs   monday morning otw  wor...
31960    user sikh temple vandalised   calgary wso cond...
31961                                 thank  user   follow
Name: tweet, Length: 31962, dtype: object

# Handling Emojis

In [37]:
import re
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               u"\U00002702-\U00002780"
                               u"\U000024C2-\U0001F251"
                               "]+",flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)

In [38]:
remove_emojis("Loved the movie,It was just 😍😍😘")

'Loved the movie,It was just '

In [39]:
import emoji
print(emoji.demojize("Loved the movie,It was just 😍😍😘"))

Loved the movie,It was just :smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::face_blowing_a_kiss:


# Tokenization
1. using the split function

In [40]:
#word tokenization

sent1 = 'I am new in New Delhi'
sent1.split()

['I', 'am', 'new', 'in', 'New', 'Delhi']

In [41]:
# sentence tokenization

sent1 = 'I am working in London. I love eating fruits. We will go to Banglore'
sent1.split('.')

['I am working in London', ' I love eating fruits', ' We will go to Banglore']

2.Regular Expression

In [42]:
import re
sent3 = 'I am going to Agra!'
tokens = re.findall("[\w']+",sent3)
tokens

['I', 'am', 'going', 'to', 'Agra']

In [43]:
text = """Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. 
Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. 
Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. 
Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
"""
sentences = re.compile('[.!?]').split(text)
sentences

['Lorem ipsum dolor sit amet, consectetur adipiscing elit',
 ' Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua',
 ' \nUt enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat',
 ' \nDuis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur',
 ' \nExcepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum',
 '\n']

3.NLTK

In [44]:
from nltk.tokenize import word_tokenize,sent_tokenize

In [45]:
sent3 = "I have a PHd in AI!@#"
sent4 = "we are here to help ! mail us at hhkjshks@gmail.com"
sent5= " A 5km ride cost $90.3"


In [46]:
word_tokenize(sent3)

['I', 'have', 'a', 'PHd', 'in', 'AI', '!', '@', '#']

In [47]:
word_tokenize(sent4)

['we',
 'are',
 'here',
 'to',
 'help',
 '!',
 'mail',
 'us',
 'at',
 'hhkjshks',
 '@',
 'gmail.com']

In [48]:
word_tokenize(sent5)

['A', '5km', 'ride', 'cost', '$', '90.3']

4. Spacy

In [49]:
import spacy
nlp=spacy.load('en_core_web_sm')

In [50]:
doc1 = nlp(sent1)
doc2 = nlp(sent3)
doc3 = nlp(sent4)
doc4 = nlp(sent5)

In [51]:
for token in doc4:
    print(token)

 
A
5
km
ride
cost
$
90.3


4.Stemming and Lemitization

In [52]:
from nltk.stem.porter import PorterStemmer

In [53]:
ps = PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [54]:
sample = "walk walks walking walked"
stem_words(sample)

'walk walk walk walk'

In [55]:
text = """Time is a thing that has no beginning and no end. It is the most practical and useful thing. You cannot control the time, you cannot force it to go at a slower pace, and you cannot speed it up. It will work at its pace. Children can learn a great deal about it by writing an essay on the value of time for classes 1, 2 and 3."""
print(text)

Time is a thing that has no beginning and no end. It is the most practical and useful thing. You cannot control the time, you cannot force it to go at a slower pace, and you cannot speed it up. It will work at its pace. Children can learn a great deal about it by writing an essay on the value of time for classes 1, 2 and 3.


In [56]:
stem_words(text)

'time is a thing that ha no begin and no end. It is the most practic and use thing. you cannot control the time, you cannot forc it to go at a slower pace, and you cannot speed it up. It will work at it pace. children can learn a great deal about it by write an essay on the valu of time for class 1, 2 and 3.'

Lemitization

In [57]:
import nltk 
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

sentence = "he was running and eating at same time. He has a bad habit of swimming after playing long hours in the sun."
punctuation = "?:!,.;"
sentence_words = nltk.word_tokenize(sentence)
for word in sentence_words:
    if word in punctuation:
        sentence_words.remove(word)
sentence_words
print("{0:20}{1:20}".format("Word","Lemma"))
for word in sentence_words:
    print("{0:20}{1:20}".format(word,wordnet_lemmatizer.lemmatize(word,pos='v')))

Word                Lemma               
he                  he                  
was                 be                  
running             run                 
and                 and                 
eating              eat                 
at                  at                  
same                same                
time                time                
He                  He                  
has                 have                
a                   a                   
bad                 bad                 
habit               habit               
of                  of                  
swimming            swim                
after               after               
playing             play                
long                long                
hours               hours               
in                  in                  
the                 the                 
sun                 sun                 
